In [ ]:
import holodeck as holo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from holodeck import utils, cosmo
from holodeck import plot as hplot
import holodeck.accretion
from holodeck.constants import MSOL, PC, YR, MPC, GYR, MYR
import gwb as psgwb
# import seaborn as sns

holo.log.setLevel(holo.log.INFO)

In [ ]:
ECCEN_INIT = 0.5
F_EDD = 1.0
SELECT = None

pop = holo.population.Pop_Illustris(select=SELECT)
size = pop.size
eccen = np.ones(size) * ECCEN_INIT
pop = holo.population.Pop_Illustris(eccen=eccen, select=SELECT)

# redz = cosmo.a_to_z(pop.scafa)

hards = [
    holo.hardening.Hard_GW,
    holo.hardening.CBD_Torques(allow_softening=True),
    holo.hardening.Sesana_Scattering(),
    holo.hardening.Dynamical_Friction_NFW(attenuate=True),
]

acc = holo.accretion.Accretion(
    accmod='Siwek22', f_edd=F_EDD, subpc=True, evol_mass=True, edd_lim=1.0,
)

In [ ]:
evo = holo.evolution.New_Evolution(pop, hards, acc=acc)
evo.evolve()

In [ ]:
fobs_cents, fobs_edges = utils.pta_freqs(num=15)
vals = evo.at('fobs', fobs_cents)

In [ ]:
gwb = evo.gwb(fobs_edges, nharms=7, nreals=30)

In [ ]:
hc = np.median(np.sqrt(gwb), axis=-1)

In [ ]:
fig, ax = holo.plot.figax()
for ii in range(min(hc.shape[1], 6)):
    ax.plot(fobs_cents, hc[:, ii], label=ii+1)
    
ax.plot(fobs_cents, np.sqrt(np.median(np.sum(gwb, axis=1), axis=1)), 'k--')
plt.legend(title='harmonic')
plt.show()